# 🎓 Building a Sentiment Classifier with Logistic Regression

In this notebook, we will build a **Logistic Regression** classifier to predict the sentiment (positive or negative) of customer reviews for women's clothing from an e-commerce website. 

For this task, we will use the **LogisticRegression** class from the scikit-learn library.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
from google.cloud import aiplatform

In [4]:
file_path = 'reviews.csv'
df = pd.read_csv(file_path)
print("✅ Successfully loaded the dataset.")
print("Dataset preview:")
print(df.head())

✅ Successfully loaded the dataset.
Dataset preview:
                                         Review Text  sentiment
0  Absolutely wonderful - silky and sexy and comf...          1
1  Love this dress!  it's sooo pretty.  i happene...          1
2  I love, love, love this jumpsuit. it's fun, fl...          1
3  This shirt is very flattering to all due to th...          1
4  I love tracy reese dresses, but this one is no...         -1


In [5]:
# Get some basic information about the dataset
print("\nDataset Information:")
df.info()


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19818 entries, 0 to 19817
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  19818 non-null  object
 1   sentiment    19818 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 309.8+ KB


In [6]:
# --- Step 2: Split Data into Training and Testing Sets ---
# It's crucial to test our model on data it has never seen before.
# We'll use 80% of the data for training and 20% for testing.
X = df['Review Text']
y = df['sentiment']

# 'stratify=y' ensures that the proportion of positive and negative reviews is the same in both your training set and your testing set.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"\nData split into {len(X_train)} training samples and {len(X_test)} testing samples.")


Data split into 15854 training samples and 3964 testing samples.


In [7]:
# --- Step 3: Feature Engineering with Bag-of-Words ---
# Here, we convert the text reviews into numerical feature vectors.
# Each feature is a count of how many times a word appears in a review.
print("\nConverting text to numerical features using Bag-of-Words...")

# Initialize the vectorizer. `stop_words='english'` removes common
# English words like 'the', 'a', 'is', which don't carry much sentiment.
vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizer on the TRAINING data and transform it into a matrix
X_train_bow = vectorizer.fit_transform(X_train)

# ONLY transform the TESTING data using the already-fitted vectorizer
X_test_bow = vectorizer.transform(X_test)

print("✅ Text successfully converted to feature vectors.")


Converting text to numerical features using Bag-of-Words...
✅ Text successfully converted to feature vectors.


In [8]:
# --- Step 4: Train the Linear Classifier ---
# We'll use Logistic Regression, a reliable linear model for classification.
print("\nTraining the Logistic Regression model...")

# Initialize the model
# max_iter is increased to ensure the model has enough time to find the best weights
model = LogisticRegression(max_iter=2000)

# Train the model on our Bag-of-Words training data
model.fit(X_train_bow, y_train)

print("✅ Model training complete!")


Training the Logistic Regression model...
✅ Model training complete!


In [9]:
# --- Step 5: Evaluate the Model's Performance ---
# Let's see how accurately our model predicts sentiment on the unseen test data.
print("\nEvaluating model performance on the test set...")

# Make predictions on the test data
y_pred = model.predict(X_test_bow)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"📈 Model Accuracy: {accuracy:.4f} ({accuracy:.2%})")


Evaluating model performance on the test set...
📈 Model Accuracy: 0.9294 (92.94%)


In [10]:
# --- Step 6: Predict Sentiment on New Reviews ---
# This is the fun part! Let's use our fine-tuned model on brand new text.
print("\n--- Making Predictions on New Reviews ---")

new_reviews = [
    "This dress is absolutely beautiful and fits perfectly!",
    "The material felt cheap and it was not what I expected.",
    "It's an okay product, not great but not terrible either.",
    "I am so disappointed with this purchase, I will be returning it."
]

# 1. Transform the new reviews into the Bag-of-Words format
new_reviews_bow = vectorizer.transform(new_reviews)

# 2. Predict using our trained model
new_predictions = model.predict(new_reviews_bow)


for i in range(len(new_reviews)):
    print(f"Review: {new_reviews[i]}")
    print(f"Predicted Sentiment: {new_predictions[i]}\n")


--- Making Predictions on New Reviews ---
Review: This dress is absolutely beautiful and fits perfectly!
Predicted Sentiment: 1

Review: The material felt cheap and it was not what I expected.
Predicted Sentiment: -1

Review: It's an okay product, not great but not terrible either.
Predicted Sentiment: -1

Review: I am so disappointed with this purchase, I will be returning it.
Predicted Sentiment: -1



In [11]:
joblib.dump(model, 'model.joblib')

['model.joblib']

In [22]:
aiplatform.init(project='tidy-visitor-470603-g4', location='us-central1')

endpoint = aiplatform.Endpoint(
    endpoint_name='1564265297232265216'
)

print('Successfully created Endpoint object')
print("Endpoint name:" + endpoint.resource_name)

Successfully created Endpoint object
Endpoint name:projects/795116648083/locations/us-central1/endpoints/1564265297232265216


In [ ]:
new_review = ["The material felt cheap and it was not what I expected."]
print(f"New review to classify: '{new_review[0]}'" )

New review to classify: 'The material felt cheap and it was not what I expected.'


In [28]:
sparse_matrix = vectorizer.transform(new_review)
numpy_array = sparse_matrix.toarray()
processed_review = numpy_array.tolist()

print("\nFinal list ready for prediction:", processed_review)

print("\nSending prediction request to the Vertex AI endpoint...")
response = endpoint.predict(instances=processed_review)

print(response)
print(type(response))



Final list ready for prediction: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [30]:
prediction_result = response.predictions[0]

sentiment = "Positive" if prediction_result == 1 else "Negative"

print("---------------------------------------------------")
print(f"Prediction: The review sentiment is '{sentiment}'")
print("---------------------------------------------------")

---------------------------------------------------
Prediction: The review sentiment is 'Negative'
---------------------------------------------------


In [ ]:
new_reviews = [
    "I am so disappointed with this purchase. I will be returning it",
    "The material felt cheap and it was not what I expected",
    "It's an okay product, not great but not terriable either",
    "This dress is absolutely beautiful and fits perfectly!",
    "I recently purchased this dress and I have to say, it exceeded my expectations."
]

sparse_matrix = vectorizer.transform(new_reviews)
numpy_array = sparse_matrix.toarray()
processed_review = numpy_array.tolist()

print("\nFinal list ready for prediction:", processed_review)

print("\nSending prediction request to the Vertex AI endpoint...")
response = endpoint.predict(instances=processed_review)

print(response)
print(type(response))

prediction_result = response.predictions[0]

sentiment = "Positive" if prediction_result == 1 else "Negative"

print("---------------------------------------------------")
print(f"Prediction: The review sentiment is '{sentiment}'")
print("---------------------------------------------------")